In [2]:
from google.colab import drive #Підключаємо Диск
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd #Завантажуємо файли

file_path_sh = '/content/drive/MyDrive/Colab_Notebooks/survey_results_schema.csv' #Якого беня він називається 'My Drive' а при посиланні 'MyDrive'?
file_path = '/content/drive/MyDrive/Colab_Notebooks/survey_results_public.csv'
df_sh = pd.read_csv(file_path_sh)
df = pd.read_csv(file_path)

In [4]:
count_active_responds = df['ResponseId'].count() #Підрахунок непустих респондентів.

In [5]:
df_cleaned = df.dropna() #Перевірка наявності усіх відповідей.
complete_responds=len(df_cleaned)

In [6]:
df_work_exp=df.dropna(subset=['WorkExp']) #Розрахунок мір центральної тенденції, для початку дропнем людей що не вказали досвід.
exp_avg = df_work_exp['WorkExp'].mean()
exp_median = df_work_exp['WorkExp'].median()
exp_moda = df_work_exp['WorkExp'].mode()[0]

In [7]:
remote_df=df.loc[df['RemoteWork'] == 'Remote'] #Рахуємо кількіть записів по відаленій роботі.
remote_num=len(remote_df)

In [8]:
python_count = df['LanguageHaveWorkedWith'].str.contains('Python', case=False, na=False).sum() #Рахуємо кількість Пайтон програмерів, використовуємо ігнорування кейсу. Робимо напряму без нового дейтафрейму.


In [9]:
online_count = df['LearnCodeOnline'].notna().sum() #рахуємо кількість респондентів що навчались онлайн

In [10]:
filtered_from_na = df.dropna(subset=['ConvertedCompYearly']) #Тут чищу від НА бо вже боюсь нижньої строки
avg_comp_by_country = filtered_from_na.groupby('Country')['ConvertedCompYearly'].agg(Average_Compensation='mean', Median_Compensation ='median').reset_index()
#групуємо Пайтон програмістів по країнах та рахуємо компенсації по типам
avg_comp_by_country['Average_Compensation']=avg_comp_by_country['Average_Compensation'].round(2) #залишаю по 2 символи "шоб красіво".
avg_comp_by_country['Median_Compensation']=avg_comp_by_country['Median_Compensation'].round(2)

In [11]:
hi_comp=df.sort_values('ConvertedCompYearly', ascending=False) #Сортуємо компенсації по спадаючії.
hi_comp['ConvertedCompYearly'] = hi_comp['ConvertedCompYearly'].apply(lambda x: f'{x:,.0f}') #чесно стирив у ГПТенка, бо хотів відформатувати результат без дробної частини та з роздільником тисяч.

In [12]:
python_filter = df['LanguageHaveWorkedWith'].str.contains('Python', case=False, na=False) # створюю булевий фільтр для датафрейму
users_per_group = df.groupby('Age').size() # Группування по віку - новий датафрейм
python_users_per_group = df[python_filter].groupby('Age').size() #накладаємо фільтр на новий датафрейм
python_percentage = python_users_per_group / users_per_group
percentage_df = pd.DataFrame(python_percentage, columns=['Python_User_%']) #створюємо датафрейм з результатів розрахунку
percentage_df['Python_User_%'] = percentage_df['Python_User_%'].apply(lambda x: f'{x:,.2%}') #форматування результату ділення у "красівий" процент

In [13]:
quant = df['ConvertedCompYearly'].quantile(0.75) # Створюю маску для 3 квартилю
quantile_df = df[df['ConvertedCompYearly'] >= quant] # Накладаю маску на оригінальний ДФ
remote_mask = quantile_df['RemoteWork'].str.contains('Remote', case=False, na=False) #Сворюю маску для віддалених
quantile_remote_df = quantile_df[remote_mask] #Накладаю маску на МОДИФІКОВАНИЙ!!! ДФ, отримуємо фінальний зріз, згідно вимог
top_ind = quantile_remote_df['Industry'].value_counts() #Підрахунок значень по колонці Індустрія

In [44]:
print('1. Кількість активних респондентів -', count_active_responds)
print('\n2. Кількість респондентів що відповіли на усі питання -', complete_responds)
print(f'\n3. Міри центральної тенденції для досвіду роботи:\n\n    Середнє - {exp_avg:.2f}\n    Медіана - {exp_median}\n    Мода - {exp_moda}')
print('\n4. Кількість респондентів що працюють віддалено -',remote_num)
print(f'\n5. Доля респондентів що програмують на Python - {python_count/count_active_responds:.2%}')
print('\n6. Кількість респондентів, що навчилися програмувати за допомогою онлайн -',online_count)
print('\n7. Середня та медіанна сума компенсації по країнах. Перша десятка\n\n', avg_comp_by_country.head(10).to_string(index=False))
print('\n 8. Рівні освіти у найбагатших програмістів:\n\n', hi_comp[['ConvertedCompYearly','EdLevel']].head(5))
print('\n----------------------------------------------------------------------\n')
print('\n9.Бонус 1. Користувачі Пайтона по віковим групам:\n\n', percentage_df.reset_index().to_string(index=False))
print('\n10. Бонус 2. Найпопулярніші індустрії серед найбагатших ледачих програмістів:\n\n', top_ind)

1. Кількість активних респондентів - 65437

2. Кількість респондентів що відповіли на усі питання - 0

3. Міри центральної тенденції для досвіду роботи:

    Середнє - 11.47
    Медіана - 9.0
    Мода - 3.0

4. Кількість респондентів що працюють віддалено - 20831

5. Доля респондентів що програмують на Python - 47.06%

6. Кількість респондентів, що навчилися програмувати за допомогою онлайн - 49237

7. Середня та медіанна сума компенсації по країнах. Перша десятка

             Country  Average_Compensation  Median_Compensation
        Afghanistan               5057.25               3438.0
            Albania              39207.55              35341.0
            Algeria              91836.70               7120.0
            Andorra             123517.00             123517.0
             Angola               1366.33               1754.0
Antigua and Barbuda             126120.00             126120.0
          Argentina              43656.84              30000.0
            Armenia      